In [1]:
import re 
from collections import defaultdict, deque
from functools   import lru_cache
from typing      import Dict, Tuple

from helpers     import data 

In [2]:
rules = data(7)
rules[:3]

['shiny purple bags contain 2 pale blue bags, 1 wavy fuchsia bag, 5 pale salmon bags.',
 'bright gray bags contain 4 dotted coral bags.',
 'clear chartreuse bags contain 3 dark magenta bags, 3 dull gray bags, 4 dark silver bags.']

**Part 1:** Count the number of bags that can ultimately contain a gold bag. Find all bags that directly contain gold bag, then repeat for those bags to find other deeper ancestors of a gold bag. All ancestors are valid. 

In [3]:
# First two words are holder bag name
# Then search for "2 shiny gold bag[s]": digit, space, name, space, bags?
get_holder = re.compile(r"^([^\s]*\s[^\s]*)")
def holds(name):
    return re.compile(r"\d+\s[^\s]*" + name + r"\s(bag)s?")

In [4]:
get_holder.search("shiny purple bags contain 2 pale blue bags, 1 wavy fuchsia bag, 5 pale salmon bags").group()

'shiny purple'

In [5]:
holders = set()
can_hold = ["shiny gold"]
while can_hold: 
    child = re.compile(can_hold.pop())
    for rule in rules: 
        if child.search(rule): 
            holder = get_holder.search(rule).group()
            if holder not in holders and holder != "shiny gold": 
                holders.add(holder)
                can_hold.append(holder)
len(holders)

179

**Part 1, Take 2:** 

In [6]:
# First two words are holder bag name
GET_HOLDER = re.compile(r"^([^\s]*\s[^\s]*)")

# Held bag is found as 12 shiny gold bags, ie digits, space, name, space, bag(s)
holds = lambda name: re.compile(f"\d*\s({name})\sbags?")

possible_holders = set()
possible_children = deque(["shiny gold"])

while possible_children: 
    child = possible_children.pop()
    HOLDS_CHILD = holds(child)
    for rule in rules: 
        if HOLDS_CHILD.search(rule):
            holder = GET_HOLDER.search(rule).group(1)
            if holder not in possible_holders and holder != "shiny_gold":
                possible_holders.add(holder)
                possible_children.append(holder)
len(possible_holders)

179

**Norvig:** Norvig's solution is way better. Also, I forgot to use `assert`s to do simple tests!

In [7]:
Bag = str 
BagRules = Dict[Bag, Dict[Bag, int]] # {outer: {inner: count, ...}, ...} since can hold multiple types of bags 

def parse_inner(text: str) -> Tuple[Bag, int]:
    """
    Get (bag: count) for a single bag
    
    >>> parse_inner("3 muted gray")
    ('muted gray', 3)
    
    >>> parse_inner("no other")
    ('other', 0)
    """
    n, bag = text.split(maxsplit=1) # Split only one first space 
    return bag, 0 if n == "no" else int(n)

def parse_bag_rule(line: str) -> Tuple[Bag, Dict[Bag, int]]: 
    """
    Return a single rule: (outer, {inner: count, ...})
    
    >>> parse_bag_rule("shiny plum bags contain 4 pale blue bags, 5 dull brown bags, 5 mirrored black bags.")
    ('shiny plum', {'pale blue': 4, 'dull brown': 5, 'mirrored black': 5})
    
    >>> parse_bag_rule("dull bronze bags contain no other bags.")
    ('dull bronze', {'other': 0})
    """
    line = re.sub(" bags?|[.]", "", line) # Remove unnecessary info 
    outer, inner = line.split(" contain ")
    return outer, dict(map(parse_inner, inner.split(", ")))

bag_rules: BagRules = dict(map(parse_bag_rule, rules))

@lru_cache(maxsize=None)
def contains(bag, target) -> bool: 
    """Does this bag contain the target, perhaps recursively?"""
    contents = bag_rules.get(bag, {})
    return target in contents or any(contains(inner, target) for inner in contents)

sum(contains(bag, "shiny gold") for bag in bag_rules)

179

**Part 2:** How many individual bags are required inside a shiny gold bag?

In [8]:
get_bags_held = re.compile(r"\d+\s([^\s]*\s[^\s]*)\s")

def get_num_bags(name):
    return re.compile(f"(\d+)\s{name}\s(bag)s?")

In [9]:
def count_total(name): 
    total = 0
    for rule in rules: 
        if get_holder.search(rule).group() == name: 
            for held_bag in get_bags_held.findall(rule):
                times = int(get_num_bags(held_bag).search(rule).group(1))
                total += times 
                total += times * count_total(held_bag)
    return total 
            
count_total("shiny gold")        

18925

**Norvig:**

In [10]:
def num_contained_in(target, rules) -> int: 
    """How many bags are contained (recursively) in target?"""
    return sum(n + n * num_contained_in(bag, rules)
               for (bag, n) in rules[target].items() if n > 0)

num_contained_in("shiny gold", bag_rules)

18925